# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}\text{loss}(r_{ij}, \tilde r_{ij}) + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$
* $\text{loss}$ is mean squared error

In [1]:
const name = "ExplicitUserItemBiases"
const implicit = false;

In [2]:
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");
@nbinclude("ExplicitUserItemBiasesBase.ipynb");

In [3]:
const training = get_split("training", implicit)
const validation = get_split("validation", implicit);

## Alternating Least Squares
* Given some hyperparameters $\lambda$, we can solve for $U$ and $A$ via Alternating Least Squares
* This is an iterative algorithm where we fix $A$, then solve for the $U$ that minimizes the loss function
* Then we fix $U$ and solve for the best $A$
* These two steps are repeated until the matrices $U$ and $A$ converge
### More details
* If we fix $a$, then for each user $i$, $u_i$ is optimized when
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [ ]:
function train_model(training, stop_criteria, λ)
    @info "training model with parameters $λ"
    λ_u, λ_a, λ_wu, λ_wa = λ
    users, items, ratings = training.user, training.item, training.rating
    weights =
        expdecay(get_counts("training", implicit), log(λ_wu)) .*
        expdecay(get_counts("training", implicit; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), num_users())
    a = zeros(eltype(λ_a), num_items())

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

In [7]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ)
    r = make_prediction(validation.user, validation.item, u, a)
    residualized_loss([], implicit, r)
end;

In [8]:
# Find the best regularization hyperparameters
res = Optim.optimize(
    validation_mse,
    fill(0.0f0, 4),
    Optim.BFGS(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220621 21:01:16 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,0.0,1.0)]
Progress: 100%|███████████████████████████| Time: 0:00:00 (53.04 ns/it)39m


Iter     Function value   Gradient norm 
     0     1.778537e+00     7.680463e-02
 * Current step size: 1.0
 * time: 0.008833169937133789
 * g(x): Float32[-0.0065774093, -1.8496149f-6, 0.019703476, 0.07680463]
 * ~inv(H): Float32[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
 * x: Float32[0.0, 0.0, 0.0, 0.0]


[ Info: 20220621 21:01:47 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0065991,1.0065991,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000019,0.0,1.0000019,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9804894,0.0,0.0,0.9804894,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.92607075,0.0,0.0,0.0,0.92607075)]
[ Info: 20220621 21:02:10 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0039606,1.0039606,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000011,0.0,1.0000011,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9882287,0.0,0.0,0.9882287,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.95489186,0.0,0.0,0.0,0.95489186)]


     1     1.776097e+00     1.640687e-02
 * Current step size: 0.6009688
 * time: 44.82783508300781
 * g(x): Float32[-0.00197358, -2.6437733f-6, 0.016406871, 0.011619643]
 * ~inv(H): Float32[1.000341 2.7888984f-6 -0.0145523 0.010747191; 2.7888984f-6 1.0 -1.2159333f-5 -2.8423645f-5; -0.0145523 -1.2159333f-5 1.0841252 0.1257996; 0.010747187 -2.8423647f-5 0.12579961 0.7024923]
 * x: Float32[0.0039528175, 1.1115608f-6, -0.011841174, -0.046157185]


[ Info: 20220621 21:02:32 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0060593,1.0060593,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000043,0.0,1.0000043,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9693605,0.0,0.0,0.9693605,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9451963,0.0,0.0,0.0,0.9451963)]
[ Info: 20220621 21:02:54 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0144975,1.0144975,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.000017,0.0,1.000017,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.8974221,0.0,0.0,0.8974221,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9073885,0.0,0.0,0.0,0.9073885)]
[ Info: 20220621 2

...

     9     1.772694e+00     1.762123e-05
 * Current step size: 0.97539425
 * time: 596.2922401428223
 * g(x): Float32[1.859895f-6, 1.987044f-6, -3.8121045f-6, -1.7621225f-5]
 * ~inv(H): Float32[685.1778 -3.0621037 46.025127 46.72199; -3.0621035 1.0137067 -0.20475462 -0.20908837; 46.025135 -0.20475464 16.575941 0.79843825; 46.721985 -0.20908837 0.79843724 3.910745]
 * x: Float32[-0.24045375, 0.0012707366, -0.38615683, -0.023525553]


[ Info: 20220621 21:11:43 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.78605914,0.78605914,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0012708,0.0,1.0012708,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.67965853,0.0,0.0,0.67965853,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9767348,0.0,0.0,0.0,0.9767348)]
[ Info: 20220621 21:12:05 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7852121,0.7852121,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0012677,0.0,1.0012677,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.67963696,0.0,0.0,0.67963696,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9766781,0.0,0.0,0.0,0.9766781)]
[ Info: 20

    10     1.772694e+00     1.990028e-06
 * Current step size: 1.0213934
 * time: 662.778510093689
 * g(x): Float32[-5.409738f-8, 1.9900276f-6, 1.15627984f-7, 8.306294f-7]
 * ~inv(H): Float32[701.3097 -0.5484613 46.069664 48.020363; -0.54846114 1.0281942 -0.13199009 -0.07236117; 46.069668 -0.1319901 16.564577 0.81346804; 48.02036 -0.07236118 0.813467 4.003841]
 * x: Float32[-0.24072903, 0.0012699357, -0.38616493, -0.02354039]


[ Info: 20220621 21:12:50 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7860497,0.7860497,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0012687,0.0,1.0012687,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.67965853,0.0,0.0,0.67965853,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.97673386,0.0,0.0,0.0,0.97673386)]
[ Info: 20220621 21:13:12 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7860303,0.7860303,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0012608,0.0,1.0012608,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.67965895,0.0,0.0,0.67965895,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.97673124,0.0,0.0,0.0,0.97673124)]
[ Info: 

    11     1.772694e+00     1.201726e-05
 * Current step size: 22.339348
 * time: 751.7949130535126
 * g(x): Float32[5.4693913f-7, 1.9903284f-6, -6.620153f-7, -1.2017262f-5]
 * ~inv(H): Float32[540.6965 43.46041 48.073666 33.137325; 43.460415 46.311863 -2.027276 5.6351824; 48.07367 -2.0272758 16.571175 0.9609119; 33.137318 5.635182 0.96091086 2.6710274]
 * x: Float32[-0.24086717, 0.0012252473, -0.38616127, -0.023555536]


In [9]:
@info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

[ Info: 20220621 21:14:19 The optimal λ is Float32[0.785946, 1.001226, 0.6796609, 0.97671974], found in 35 function calls


In [10]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ);

[ Info: 20220621 21:14:19 training model with parameters Float32[0.785946, 1.001226, 0.6796609, 0.97671974]


In [11]:
validation_mse(Optim.minimizer(res))

[ Info: 20220621 21:14:26 training model with parameters Float32[0.785946, 1.001226, 0.6796609, 0.97671974]


1.7725976f0

## Inference

In [12]:
model(users, items) = make_prediction(users, items, u, a)
write_alpha(model, [], implicit, name);

[ Info: 20220621 21:14:36 validation loss: 1.7725976, β: Float32[1.0017253]
[ Info: 20220621 21:14:39 training loss: 1.6141207, β: Float32[1.0017253]


In [13]:
write_params(Dict("u" => u, "a" => a, "λ" => λ), name);